<a href="https://colab.research.google.com/github/schnap3380/kewwords_for_journal_analysis/blob/main/papers_in_journals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import requests # for accessing API
from datetime import datetime  # for dealing with dates
from time import time

In [43]:
# Semantic Scholar API key (by request - mine allows for 100 requests / second)
headers={'x-api-key':'FCndxzhW160dParwnevD46jKxnLLuBv7DE3UR1qa'}

In [68]:
def get_articles_from_journal(journal_name, api_key):
    # Endpoint for searching articles in Semantic Scholar
    url = "https://api.semanticscholar.org/graph/v1/paper/search/bulk"

    # Prepare the search parameters
    params = {
        "query": f'journal:"{journal_name}"',  # Searching by journal name
        "apiKey": api_key
    }

    # Send request to Semantic Scholar API
    response = requests.get(url, params=params)

    # Check if the request was successful
    if response.status_code != 200:
        raise Exception(f"Error fetching data from Semantic Scholar: {response.status_code}")

    if response.status_code == 429:
        print("Waiting 1 second for access to the API...")
        time.sleep(1)
        response = requests.get(url, params=params)


    # Parse JSON response
    data = response.json()

    # Extract the relevant details for each paper
    articles = []
    for paper in data['data']:
        title = paper.get('title', '')
        abstract = paper.get('abstract', '')
        year = paper.get('year', 'Unknown')

        articles.append([title, abstract, year])

    # Convert the data into a pandas DataFrame
    df = pd.DataFrame(articles, columns=['Title', 'Abstract', 'Year'])

    return df , data

In [78]:
df , data= get_articles_from_journal("Nature",headers)


In [75]:
import requests
import pandas as pd

def fetch_articles_from_journal(api_key, journal_name):
    url = "https://api.semanticscholar.org/graph/v1/paper/search/bulk"
    params = {
        "query": journal_name,
        "fields": "title,abstract,year",
        "apiKey": api_key
    }

    try:
        response = requests.get(url, params=params)

        if response.status_code != 200:
          raise Exception(f"Error fetching data from Semantic Scholar: {response.status_code}")
        if response.status_code == 429:
          print("Waiting 1 second for access to the API...")
          time.sleep(1)
          response = requests.get(url, params=params)

        response.raise_for_status()  # Raises an HTTPError for bad responses
        data = response.json()

        if 'data' not in data:
            return pd.DataFrame()  # Return an empty DataFrame if no data is found

        articles = [{
            'Title': article.get('title', ''),
            'Abstract': article.get('abstract', ''),
            'Year': article.get('year', '')
        } for article in data.get('data', [])]

        return pd.DataFrame(articles)

    except requests.RequestException as e:
        print(f"Request failed: {e}")
        return pd.DataFrame()  # Return an empty DataFrame in case of an error

# Example usage:
api_key = 'FCndxzhW160dParwnevD46jKxnLLuBv7DE3UR1qa'
journal_name = 'Nature'
df = fetch_articles_from_journal(api_key, journal_name)
print(df)


                                                 Title  \
0    De la nature de la «transition»: remarques épi...   
1                           THE SUBJECT OF SOVEREIGNTY   
2                Changes in Coping with Chronic Stress   
3    Global Value Chains and Changing Trade Elastic...   
4    Critical health legislation: a blueprint for a...   
..                                                 ...   
995  An Approach to Generate Music using Genetic Al...   
996  Industrialisation, innovation et institutions ...   
997  International Journal of Innovation in Teachin...   
998                The Quest for Liberatory Psychology   
999  The first‐night effect and the consistency of ...   

                                              Abstract    Year  
0    L'effondrement du communisme vaut, sinon accen...  1994.0  
1    Seeking new forms of democracy, progressive po...  2023.0  
2                                                 None  1997.0  
3    The trade collapse of 2008-2009 and th

In [77]:
df.isna().sum()

,0
Title,0
Abstract,156
Year,6
